<a href="https://colab.research.google.com/github/nshantha/NLP/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [4]:
for data_item  in range(len(train_data)):
  vars(train_data.examples[data_item]).get('text').reverse()

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
vars(train_data.examples[1]). get('text')

['.',
 'classic',
 'a',
 'of',
 'shy',
 'just',
 'all',
 'in',
 '/>All',
 '/><br',
 'win.<br',
 'to',
 'Rob',
 'want',
 'really',
 'you',
 'and',
 'good',
 'is',
 'story',
 'The',
 '.',
 'gory',
 'too',
 'not',
 'but',
 ',',
 'realistic',
 'very',
 'play',
 'sword',
 'the',
 'and',
 'exciting',
 'is',
 'Action',
 '/>The',
 '/><br',
 'filmed.<br',
 'and',
 'acted',
 'well',
 'very',
 'is',
 'movie',
 'whole',
 'the',
 'and',
 ',',
 'strong',
 'is',
 'cast',
 'the',
 'of',
 'rest',
 '/>The',
 '/><br',
 'times.<br',
 'many',
 'too',
 '1',
 '"',
 'honour',
 '"',
 'word',
 'the',
 'used',
 'writers',
 'the',
 'think',
 'i',
 'but',
 ',',
 'too',
 'excellent',
 'is',
 ')',
 'Neeson',
 'Liam',
 '(',
 'Roy',
 '/>Rob',
 '/><br',
 'evil.<br',
 'completely',
 'look',
 'and',
 'off',
 'eyes',
 'his',
 'turn',
 'to',
 'able',
 'be',
 'to',
 'seems',
 'he',
 'face',
 'his',
 'watch',
 'you',
 'if',
 'and',
 'cold',
 'Brutally',
 '.',
 'story',
 'this',
 'in',
 'character',
 'and',
 'villain',
 'amaz

In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [95]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)

        self.lstm = nn.LSTM(hidden_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout) 
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
           
        embedded = self.dropout(self.embedding(text))        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        LSTM_LAYERS = 2
        for layer in range(LSTM_LAYERS):
          packed_output, (hidden, cell) = self.lstm(packed_output)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        hidden = self.dropout(hidden[-1,:,:])
        
            
        return self.fc(hidden)

In [96]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = False # Trying to reverse manually
DROPOUT = 0.2 # Dropout changed from 0.5->0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [97]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,393,385 trainable parameters


In [98]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [99]:

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]])

In [100]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]])


In [101]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [102]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [103]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [104]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        #text = torch.flip(text, [0]) # reversing the text
        # print('Training :  Test type {} {}'.format(type(text_lengths),text_lengths.shape))    
        predictions = model(text, text_lengths.cpu()).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    #print('Training :  Test type {} {}'.format(type(text),text.shape))    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [105]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths.cpu()).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [106]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [107]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 25s
	Train Loss: 0.651 | Train Acc: 61.85%
	 Val. Loss: 0.544 |  Val. Acc: 73.37%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.560 | Train Acc: 71.33%
	 Val. Loss: 0.523 |  Val. Acc: 73.93%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.453 | Train Acc: 80.02%
	 Val. Loss: 0.737 |  Val. Acc: 65.01%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.370 | Train Acc: 84.66%
	 Val. Loss: 0.357 |  Val. Acc: 85.23%
Epoch: 05 | Epoch Time: 0m 26s
	Train Loss: 0.313 | Train Acc: 87.47%
	 Val. Loss: 0.342 |  Val. Acc: 85.63%
Epoch: 06 | Epoch Time: 0m 25s
	Train Loss: 0.271 | Train Acc: 89.52%
	 Val. Loss: 0.354 |  Val. Acc: 86.01%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.225 | Train Acc: 91.41%
	 Val. Loss: 0.478 |  Val. Acc: 82.53%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.206 | Train Acc: 92.11%
	 Val. Loss: 0.333 |  Val. Acc: 87.09%
Epoch: 09 | Epoch Time: 0m 26s
	Train Loss: 0.177 | Train Acc: 93.58%
	 Val. Loss: 0.379 |  Val. Acc: 85.68%
Epoch: 10 | Epoch T

In [108]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.384 | Test Acc: 85.76%
